<a href="https://colab.research.google.com/github/mohadi21/Pr-diction-des-donn-es-MNIST-avec-uniquement-100-labels/blob/main/TPDeepL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras import backend as K
import numpy as np
import random
from sklearn.metrics import confusion_matrix

chargement du dataset et pretraitement

In [ ]:


num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

num_train_samples = 100
samples_per_class = int(num_train_samples/9)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
if K.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
  x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
  input_shape = (1, img_rows, img_cols)
else:
  x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
  x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
  input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y=y_train.tolist()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# only select 100 training samples 
#idxs_annot = list(range(x_train.shape[0]))
#random.seed(0)
#random.shuffle(idxs_annot)
#idxs_annot = idxs_annot[ :100 ]

idxs_annot = list()
for j in range(10):
  for i in range(10):
    ind = y.index(j)
    y[ind]=None
    idxs_annot.append(ind)

x_train_unlabeled=x_train
x_train_labeled = x_train[ idxs_annot ]
for i in idxs_annot :
  x_train_unlabeled = np.delete(x_train_unlabeled,i,axis=0)


print(x_train_labeled.shape, 'x train label')
print(y_train_labeled.shape, 'y tain label')
print(x_train_unlabeled.shape, 'x train unlabel')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(100, 28, 28, 1) x train label
(100, 10) y tain label
(59900, 28, 28, 1) x train unlabel
(59900, 10) x train unlabel


Baseline du model avec 100 données MNIST labelisé

In [ ]:
epochs = 1000
batch_size = 128

model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
optimizer='adam',
metrics=['accuracy'])

model.fit(x_train_labeled, y_train_labeled,
batch_size=batch_size,
epochs=epochs)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.2973 - accuracy: 0.1200
Epoch 2/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.2846 - accuracy: 0.1400
Epoch 3/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.2617 - accuracy: 0.1800
Epoch 4/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.2584 - accuracy: 0.2000
Epoch 5/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.1991 - accuracy: 0.3100
Epoch 6/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.1789 - accuracy: 0.3200
Epoch 7/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.1593 - accuracy: 0.3300
Epoch 8/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.1337 - accuracy: 0.4100
Epoch 9/1000
1/1 [==============================] - 0s 2ms/step - loss: 2.1300 - accuracy: 0.3300
Epoch 10/1000
1/1 [==============================] - 0s 3ms/step - loss: 2.0726 - accuracy: 0.4700
Epoch 11/1000
1/1 [

entrainement du model sur les données non labelisé

In [ ]:
new_x_train = x_train_labeled
new_y_train = y_train_labeled
listPred = list()
start = 0
for i in range(x_train_unlabeled.shape[0]):
  if i%2000 == 0 and i!=0:
    predict = keras.utils.to_categorical(listPred, num_classes)
    new_x_train = np.concatenate((new_x_train, x_train_unlabeled[start:i]), axis=0)
    new_y_train = np.concatenate((new_y_train, predict), axis=0)
    start = i
    listPred = list()
    model.fit(new_x_train, new_y_train,batch_size=128,epochs=10)
  x_pred = x_train_unlabeled[i].reshape(1, img_rows, img_cols, 1)
  listPred.append(model.predict_classes(x_pred))

Epoch 1/10
17/17 [==============================] - 0s 4ms/step - loss: 0.2625 - accuracy: 0.9095
Epoch 2/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1951 - accuracy: 0.9286
Epoch 3/10
17/17 [==============================] - 0s 4ms/step - loss: 0.2079 - accuracy: 0.9248
Epoch 4/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1608 - accuracy: 0.9381
Epoch 5/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1563 - accuracy: 0.9429
Epoch 6/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1401 - accuracy: 0.9462
Epoch 7/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1390 - accuracy: 0.9457
Epoch 8/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1169 - accuracy: 0.9562
Epoch 9/10
17/17 [==============================] - 0s 4ms/step - loss: 0.1106 - accuracy: 0.9652
Epoch 10/10
17/17 [==============================] - 0s 4ms/step - loss: 0.0962 - accuracy: 0.9657
Epoch 1/10
33/33 [=

In [ ]:
predict = keras.utils.to_categorical(listPred, num_classes)
new_x_train = np.concatenate((new_x_train, x_train_unlabeled[start:]), axis=0)
new_y_train = np.concatenate((new_y_train, predict), axis=0)


Epoch 1/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0812 - accuracy: 0.9684
Epoch 2/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0811 - accuracy: 0.9685
Epoch 3/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0789 - accuracy: 0.9701
Epoch 4/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0795 - accuracy: 0.9702
Epoch 5/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0776 - accuracy: 0.9703
Epoch 6/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0766 - accuracy: 0.9713
Epoch 7/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0790 - accuracy: 0.9691
Epoch 8/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0765 - accuracy: 0.9709
Epoch 9/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0781 - accuracy: 0.9699
Epoch 10/10
484/484 [==============================] - 2s 4ms/step - loss: 0.0762 - accuracy: 0.9710

construction du nouveau model sur les données labelisé + non labélisé predit

In [ ]:
epochs = 10
batch_size = 128
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
optimizer='adam',
metrics=['accuracy'])

model.fit(new_x_train, new_y_train,
batch_size=batch_size,
epochs=epochs,
verbose=1,)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
484/484 [==============================] - 2s 4ms/step - loss: 0.4914 - accuracy: 0.8275
Epoch 2/10
484/484 [==============================] - 2s 4ms/step - loss: 0.2134 - accuracy: 0.9179
Epoch 3/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1859 - accuracy: 0.9279
Epoch 4/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1679 - accuracy: 0.9338
Epoch 5/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1628 - accuracy: 0.9360
Epoch 6/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1548 - accuracy: 0.9387
Epoch 7/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1485 - accuracy: 0.9428
Epoch 8/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1441 - accuracy: 0.9434
Epoch 9/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1422 - accuracy: 0.9444
Epoch 10/10
484/484 [==============================] - 2s 4ms/step - loss: 0.1384 - accuracy: 0.9449